<a href="https://colab.research.google.com/github/lgagne413/cle_guard/blob/main/de_work_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Welcome!

Congratulations and thank you for making it this far in the interview process!

As it should have been described to you, the following challenge is designed to reflect a real-world backend data Guardians' project.

To get started run the code cell below.


In [133]:
!wget -q https://github.com/cle-guardians-de/data-work-sample/raw/main/bball-sqlite.zip -O /content/bball-sqlite.zip
!unzip -o /content/bball-sqlite.zip
!wget -q https://github.com/cle-guardians-de/data-work-sample/raw/main/example_play.json -O /content/example_play.json

import pandas as pd
import sqlite3
import json





Archive:  /content/bball-sqlite.zip
  inflating: bball-sqlite.db         


In [143]:
cnxn = sqlite3.connect('bball-sqlite.db')
cursor = cnxn.cursor()

print("Tables available are...")
query = "SELECT name FROM sqlite_master WHERE type='table';"
df = pd.read_sql_query(query, cnxn)
print(df.head(20))

Tables available are...
                   name
0               Batting
1              Pitching
2                  Team
3                Player
4  MergedPositionalData


***The cell above should of downloaded needed files and resulted in final output displaying 4 tables from a sqlite database.  Batting, Pitching, Teams, and Players.  Please answer the following question 3 questions to the best of your ability.***

**1)** Analyst would like to see Shane Bieber's (Cleveland Player, player id=669456) earned run avg (ERA) for major league for each regular season. Along with that in the same result set they would like to see the difference between the previous seasons ERA to gauge how he's trending. Result set should be player, season, era, previous era, difference between previous year era, earned runs, and outs.  Any value that's represented as era should be in 3 decimal point precision.Pitching er=earned runs https://www.mlb.com/glossary/standard-stats/earned-run-average

In [136]:
query = """select * from Pitching where player_id = 669456 order by seasonRaw"""
df = pd.read_sql(query, cnxn)
print(df.columns)
query = """select * from Player where player_id = 669456"""
df = pd.read_sql(query, cnxn)
print(df.columns)
query = """select * from Batting where player_id = 669456"""
df = pd.read_sql(query, cnxn)
print(df.columns)
query = """select * from Team """
df = pd.read_sql(query, cnxn)
print(df.columns)


Index(['player_id', 'player', 'team_id', 'seasonRaw', 'season', 'game_type',
       'organization_id', 'w', 'l', 'g', 'gs', 'qs', 'cg', 'gf', 'sho', 'sv',
       'svo', 'outs', 'h', 'ab', 'pa', 'r', 'er', 'd', 't', 'hr', 'sac', 'sf',
       'hb', 'bb', 'ibb', 'so', 'wp', 'bk', 'no_hitter', 'one_hitter',
       'two_hitter', 'three_hitter', 'combo_shutout', 'sb', 'cs', 'ci', 'go',
       'ao', 'gidp', 'np', 's', 'hld', 'ir', 'irs', 'bq', 'bqs', 'rs',
       'first_date', 'last_date'],
      dtype='object')
Index(['player_id', 'name_last', 'name_first', 'name_middle', 'name_use',
       'name_matrilineal', 'gender', 'birth_date', 'birth_city', 'birth_state',
       'birth_country', 'death_year', 'death_month', 'death_day', 'death_date',
       'death_city', 'death_state', 'death_country', 'high_school', 'college',
       'height_feet', 'height_inches', 'weight', 'throws', 'bats',
       'pro_debut_year', 'pro_debut_month', 'pro_debut_day', 'pro_debut_date',
       'primary_sport_code', '

In [137]:
print("answer 1)")
query = """select name_last || ',' || name_first as 'player',p1.season,
round(27.0*sum(p1.er)/sum(p1.outs),3) as 'era',round(27.0*sum(p2.er)/sum(p2.outs),3) as 'prev_era',
round(27.0*sum(p1.er)/sum(p1.outs),3)-round(27.0*sum(p2.er)/sum(p2.outs),3) as 'diff_era',
sum(p1.er) as 'er',sum(p1.outs) as 'outs'
from Pitching p1 
inner join Pitching p2 on p1.season=p2.season+1 and p1.player_id=p2.player_id
inner join Player pl on p1.player_id=pl.player_id
where p1.player_id = 669456
group by pl.name_use,p1.season
"""
df = pd.read_sql(query, cnxn)
print(df.head(100))

answer 1)
         player  season    era  prev_era  diff_era   er  outs
0  Bieber,Shane    2017  2.856     0.375     2.481   55   520
1  Bieber,Shane    2018  3.288     2.856     0.432  213  1749
2  Bieber,Shane    2019  3.159     3.288    -0.129  252  2154
3  Bieber,Shane    2020  2.108     3.159    -1.051   63   807
4  Bieber,Shane    2021  3.300     2.108     1.192  132  1080


**2)** There's a new feed of player position data... we've been sent a sample play from the league. These files consist of nested json and we'd like to consume the data to begin getting this feed incorporated into our warehouse.  First step you've been tasked with is getting the positional data into sql rows/columns.  So, create a table called MergedPositionalData that consists of.. playId, timestamp, id, typ, x, y, z.  The json files is called example_play.json.  And the positional data is nested under mergedPositionalData key.  playId is under Play.  For context... this data is the x,y,z coordinate on the field of play for players, umpires, and the ball every .033 seconds.


In [150]:
print("starting 2)")
cnxn.execute("drop table if exists MergedPositionalData;")
with open('example_play.json') as j:
  data = json.load(j)  
print(data.keys())
print(data['mergedPositionalData'][0])
print(data['mergedPositionalData'][1])
print(data['mergedPositionalData'][0]['positions'][0])


df_data={'playId':[],'timestamp':[],'id':[],'type':[],'x':[],'y':[],'z':[],}
play_id = data['Play']['playId']
for tsi in range(len(data['mergedPositionalData'])):
  ts=data['mergedPositionalData'][tsi]
  for posi in range(len(ts['positions'])):
    pos = ts['positions'][posi]

    df_data['playId'].append(play_id)
    df_data['timestamp'].append(ts['timeStamp'])

    if 'id' not in pos.keys():
      df_data['id'].append(0)
    else:
      df_data['id'].append(pos['id'])
    df_data['type'].append(pos['typ'])
    df_data['x'].append(pos['x'])
    df_data['y'].append(pos['y'])
    df_data['z'].append(pos['qi'])
df=pd.DataFrame.from_dict(df_data)
#print(df.head(10))
df.to_sql('MergedPositionalData',cnxn)

starting 2)
dict_keys(['Play', 'mergedPositionalData', 'gameEvent', 'ballPositionalData', 'playEvents', 'gameMetaData', 'measurements'])
{'timeStamp': '2021-04-27T22:53:10.537', 'positions': [{'oo': 0, 'x': -11.379039063602816, 'qi': 0, 'y': 161.56143950775393, 'typ': 2, 'id': 16}, {'oo': 0, 'x': 87.40255331551018, 'qi': 0, 'y': 92.80044305786649, 'typ': 2, 'id': 15}, {'oo': 0, 'x': -100.73931207626782, 'qi': 0, 'y': 101.58825375625815, 'typ': 2, 'id': 17}, {'oo': 0, 'x': -1.467968670442033, 'qi': 0, 'y': -2.7976281571602843, 'typ': 2, 'id': 14}, {'oo': 0, 'x': 10.196311102524582, 'qi': 0, 'y': 104.21368921515476, 'typ': 1, 'id': 6}, {'oo': 0, 'x': 24.704141676368593, 'qi': 0, 'y': 35.0912115150846, 'typ': 1, 'id': 1}, {'oo': 0, 'x': 46.33992652361183, 'qi': 0, 'y': 42.76105101928655, 'typ': 3, 'id': 18}, {'oo': 0, 'x': 42.69370698593863, 'qi': 0, 'y': 91.42190261594243, 'typ': 1, 'id': 4}, {'oo': 0, 'x': -90.22524146032416, 'qi': 0, 'y': 36.699031575991725, 'typ': 3, 'id': 19}, {'oo':

In [148]:
query = """select * from MergedPositionalData"""
df = pd.read_sql(query, cnxn)
print(df.head(10))

   index                                playId                timestamp  id  \
0      0  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537  16   
1      1  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537  15   
2      2  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537  17   
3      3  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537  14   
4      4  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537   6   
5      5  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537   1   
6      6  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537  18   
7      7  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537   4   
8      8  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537  19   
9      9  a5c4cf6a-36c5-4ca9-99eb-0130a61ce917  2021-04-27T22:53:10.537   3   

   type           x           y  z  
0     2  -11.379039  161.561440  0  
1     2   87.402553   92.800443  0  
2     2 -100.739312

**3)** With the sample data you just loaded... and given that the data spec says the capture time for every object is every 1/30th of a second.  Verify that this one plays timeseries is not missing data for any of the people it contains (people are id > 0, null id's are the ball and can be ignored).
 
Return any data frame you see fit to explain... 

In [159]:
print("answer 3)")
query = """select *
from MergedPositionalData 
where id > 0 
"""
df = pd.read_sql(query, cnxn)
print(df.info())

#query = """select min(timestamp),max(timestamp),(julianday(max(timestamp))-julianday(min(timestamp)))*86400.0*30 as 'time_diff'
#from MergedPositionalData 
#where id > 0 
#"""
#df = pd.read_sql(query, cnxn)
#print(df)

query = """select id, count(id) 
from MergedPositionalData 
where id > 0 
group by id
 order by id"""
df = pd.read_sql(query, cnxn)
print(df.head(100), "\nlooks good to me")





answer 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5040 entries, 0 to 5039
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      5040 non-null   int64  
 1   playId     5040 non-null   object 
 2   timestamp  5040 non-null   object 
 3   id         5040 non-null   int64  
 4   type       5040 non-null   int64  
 5   x          5040 non-null   float64
 6   y          5040 non-null   float64
 7   z          5040 non-null   int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 315.1+ KB
None
            min(timestamp)           max(timestamp)   time_diff
0  2021-04-27T22:53:00.071  2021-04-27T22:53:10.537  313.980192
    id  count(id)
0    1        315
1    2        315
2    3        315
3    4        315
4    5        315
5    6        315
6    7        315
7    8        315
8    9        315
9   10        315
10  14        315
11  15        315
12  16        315
13  17        315
14  18        315

In [141]:
cursor.close()
cnxn.close()

##Bonus
The original instructions of this challenge stated the notebook must be idempotent.  There is a bug in this notebook if we were to re-run after it had already been run.  Find the bug and correct it.

**Once complete save the notebook to your github account (File->Save a copy in Github).**